In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf 

from baseline import Baseline
from system_t import System_T

import warnings
warnings.filterwarnings('ignore')

# Load Mixed-MNIST dataset from keras

In [2]:
from extra_keras_datasets import emnist

(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = emnist.load_data(type='balanced')
mnist_train_images = mnist_train_images.reshape((-1, 28, 28, 1)) / 255.0
mnist_test_images = mnist_test_images.reshape((-1, 28, 28, 1)) / 255.0

idx = mnist_train_labels <= 9
mnist_train_images = mnist_train_images[idx]
mnist_train_labels = mnist_train_labels[idx]

fashion = tf.keras.datasets.fashion_mnist
(fashion_train_images,fashion_train_labels), (fashion_test_images, fashion_test_labels) = fashion.load_data()

fashion_train_labels = fashion_train_labels + 10
fashion_test_labels = fashion_test_labels + 10
fashion_train_images = fashion_train_images.reshape((60000, 28, 28, 1)) / 255.0
fashion_test_images = fashion_test_images.reshape((10000, 28, 28, 1)) / 255.0

train_images = np.concatenate((mnist_train_images, fashion_train_images), axis=0)
train_labels = np.concatenate((mnist_train_labels, fashion_train_labels), axis=0)

# Bad for Water filling: a pathological setting for Water filling

In [3]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

num_class = len(np.unique(train_labels))
print("Number of slices : %d" % num_class)

y_train_one_hot = tf.keras.utils.to_categorical(train_labels)
mixed_data = (train_images, y_train_one_hot)

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

for i in range(num_class):
    if i < 10:
        data_num = 150
    else:
        data_num = 300
    
    initial_data_array.append(data_num)
    idx = np.argmax(mixed_data[1], axis=1) == i

    val_data_dict.append((mixed_data[0][idx][data_num:data_num+val_data_num], mixed_data[1][idx][data_num:data_num+val_data_num]))
    add_data_dict.append((mixed_data[0][idx][data_num+val_data_num:], mixed_data[1][idx][data_num+val_data_num:]))

    if i == 0:
        train_data = mixed_data[0][idx][:data_num]
        train_label = mixed_data[1][idx][:data_num]
        val_data = mixed_data[0][idx][data_num:data_num+val_data_num]
        val_label = mixed_data[1][idx][data_num:data_num+val_data_num]
    else:
        train_data = np.concatenate((train_data, mixed_data[0][idx][:data_num]), axis=0)
        train_label = np.concatenate((train_label, mixed_data[1][idx][:data_num]), axis=0) 
        val_data = np.concatenate((val_data, mixed_data[0][idx][data_num:data_num+val_data_num]), axis=0)
        val_label = np.concatenate((val_label, mixed_data[1][idx][data_num:data_num+val_data_num]), axis=0)   
    
train_data, train_label = shuffle(train_data, train_label)
print(train_data.shape, train_label.shape, num_class)

Number of slices : 20
(4500, 28, 28, 1) (4500, 20) 20


# Define slices

In [4]:
slice_desc = []
# Mixed mnist
a = ["Digit 0", "Digit 1","Digit 2","Digit 3","Digit 4","Digit 5","Digit 6","Digit 7","Digit 8","Digit 9",
   "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

for i in range(num_class):
    slice_desc.append('Slice: %s' % (a[i]))
    print('Slice: %s, Initial size: %s' % (a[i], initial_data_array[i]))

Slice: Digit 0, Initial size: 150
Slice: Digit 1, Initial size: 150
Slice: Digit 2, Initial size: 150
Slice: Digit 3, Initial size: 150
Slice: Digit 4, Initial size: 150
Slice: Digit 5, Initial size: 150
Slice: Digit 6, Initial size: 150
Slice: Digit 7, Initial size: 150
Slice: Digit 8, Initial size: 150
Slice: Digit 9, Initial size: 150
Slice: T-shirt/top, Initial size: 300
Slice: Trouser, Initial size: 300
Slice: Pullover, Initial size: 300
Slice: Dress, Initial size: 300
Slice: Coat, Initial size: 300
Slice: Sandal, Initial size: 300
Slice: Shirt, Initial size: 300
Slice: Sneaker, Initial size: 300
Slice: Bag, Initial size: 300
Slice: Ankle boot, Initial size: 300


# Original ( with no data acquisition ) 

In [5]:
cost_func = [1] * num_class
lr = 0.0005

ori = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
ori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 0
======= Collect Data =======
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
======= Performance =======
Loss: 0.26588 (0.00327), Average EER: 0.16637 (0.00534), Max EER: 0.65057 (0.10295)



# System T Demo on Mixed-MNIST

## Use 3000 budget, lambda=0.1, "Moderate" strategy

In [6]:
budget = 3000
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, 
                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)

======= Collect Data =======
[ 11  13  44  25  45  58  12  40  49  50  90   0  45  36 148   5 183  11
   0   0]
Total Cost: 865, Remaining Budget: 2135
======= Collect Data =======
[  5  16  96  55 117  62   7 115 150 123 219   0 425  90 169   0 347   0
  12   2]
Total Cost: 2010, Remaining Budget: 125
======= Collect Data =======
[ 0  0  0  1  0  0  8  0  3  0  0  0  0 61  0 18 35  0  0  0]
Total Cost: 126, Remaining Budget: -1

======= Performance =======
[ 16.  29. 140.  81. 162. 120.  27. 155. 202. 173. 309.   0. 470. 187.
 317.  23. 565.  11.  12.   2.]
Number of iteration: 3
Strategy: Moderate, C: 0.1, Budget: 3000
Loss: 0.21949 (0.00364), Average EER: 0.13280 (0.00605), Max EER: 0.49166 (0.08976)



# Baseline: Uniform

In [7]:
budget = 3000
uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 3000
======= Collect Data =======
[150 150 150 150 150 150 150 150 150 150 150 150 150 150 150 150 150 150
 150 150]
======= Performance =======
Loss: 0.22589 (0.00391), Average EER: 0.15657 (0.00619), Max EER: 0.59213 (0.05565)



# Baseline: Water filling

In [8]:
budget = 3000
wf = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Waterfilling')
wf.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Waterfilling, Budget: 3000
======= Collect Data =======
[225 225 225 225 225 225 225 225 225 225  75  75  75  75  75  75  75  75
  75  75]
======= Performance =======
Loss: 0.22802 (0.00292), Average EER: 0.16671 (0.00477), Max EER: 0.68417 (0.07454)

